In [1]:
import os
from pathlib import Path

In [2]:
TIMEFRAME = "15min"
PAIR = "BTCUSDT"
DATA_PATH = Path.home() / "data" / PAIR
TMP_PATH = Path("./tmp/").resolve()
CONFIG_PATH = Path("./config/").resolve()
LOG_PATH = Path("./log/").resolve()
DATA_PATH.mkdir(exist_ok=True, parents=True)
TMP_PATH.mkdir(exist_ok=True, parents=True)
CONFIG_PATH.mkdir(exist_ok=True, parents=True)
LOG_PATH.mkdir(exist_ok=True, parents=True)

Save: OHLCV + Open Interest from Bybit API

In [3]:
import ccxt
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = Path.home() / ".env"
load_dotenv(dotenv_path)
exchange = ccxt.bybit()
exchange.apiKey = os.environ["BYBIT_API_KEY"]
exchange.secret = os.environ["BYBIT_SECRET"]
exchange.options["timeDifference"] = 5000

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
ohlcv = exchange.fetch_ohlcv(PAIR, "15m", limit=200)
oi = exchange.fetch_open_interest_history(PAIR, "15min", limit=200)
oi = [[int(d["info"]["timestamp"]) * 1000, float(d["info"]["open_interest"])] for d in oi]
assert (len(ohlcv) == len(oi))
assert (ohlcv[0][0] == oi[0][0] and ohlcv[-1][0] == oi[-1][0])
data = [a + b[1:] for a, b in zip(ohlcv, oi)]
df = pd.DataFrame(data, columns=["Datetime", "Open", "High", "Low", "Close", "Volume", "OpenInterest"])
df["Datetime"] = pd.to_datetime(df["Datetime"], unit="ms")
df = df.set_index("Datetime")
df.head()

,Open,High,Low,Close,Volume,OpenInterest
Datetime,,,,,,
2022-08-19 00:45:00,22940.5,22984.0,22899.5,22928.0,2069.407,50027.152
2022-08-19 01:00:00,22928.0,22955.0,22857.5,22869.5,2475.209,50146.796
2022-08-19 01:15:00,22869.5,22895.0,22738.0,22757.0,5073.272,50324.878
2022-08-19 01:30:00,22757.0,22842.0,22745.5,22804.5,3597.629,50503.282
2022-08-19 01:45:00,22804.5,22819.0,22712.0,22755.0,2069.836,50667.028


In [6]:
from scripts.extract_features import attach_features
df = attach_features(df)
oi = df["OpenInterest"].apply(np.log1p)
for ts in [1, 5, 10, 20]:
    df[f"feature_oi_log_return_{ts}"] = oi.diff(ts)

df.head()

,Open,High,Low,Close,Volume,OpenInterest,feature_candle_value,feature_candle_value_mean_10,feature_candle_value_mean_20,feature_candle_value_mean_5,...,feature_upper_shadow_mean_20,feature_upper_shadow_mean_5,feature_volatility_10,feature_volatility_20,feature_volatility_3,feature_volatility_5,feature_oi_log_return_1,feature_oi_log_return_5,feature_oi_log_return_10,feature_oi_log_return_20
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-08-19 05:45:00,22781.0,22816.0,22766.0,22806.5,1151.719,53062.741,0.510000,-0.006954,-0.025848,0.014895,...,0.001474,0.000794,0.000807,0.001852,0.001154,0.000959,NaN,NaN,NaN,NaN
2022-08-19 06:00:00,22806.5,22812.0,22775.5,22780.5,524.017,52920.174,-0.712329,-0.074678,-0.031465,-0.023867,...,0.001357,0.000838,0.000897,0.001786,0.001304,0.001080,-0.002690,NaN,NaN,NaN
2022-08-19 06:15:00,22780.5,22781.5,22725.0,22734.5,1290.201,53089.229,-0.814159,-0.070639,-0.036345,-0.316523,...,0.001158,0.001171,0.001060,0.001471,0.001620,0.001279,0.003189,NaN,NaN,NaN
2022-08-19 06:30:00,22734.5,22749.0,21101.0,22016.0,46668.087,53214.944,-0.435983,-0.138238,-0.082755,-0.472950,...,0.002688,0.007471,0.010153,0.007277,0.017634,0.014054,0.002365,NaN,NaN,NaN
2022-08-19 06:45:00,22016.0,22016.5,21805.0,21894.5,8180.670,52993.517,-0.574468,-0.246246,-0.088348,-0.405388,...,0.002825,0.008329,0.010051,0.007326,0.016454,0.013725,-0.004170,NaN,NaN,NaN


In [7]:
from rl_bot.models.mtsfc_network import MTSFCNetwork
from ray.rllib.models import ModelCatalog
from rl_bot.envs.environment import TradingEnv
ModelCatalog.register_custom_model("MTSFCNetwork", MTSFCNetwork)

In [9]:
import json
with open(CONFIG_PATH / "trainer.json", "r") as f:
    trainer_config = json.load(f)

In [12]:
trainer_config["env_config"]["observer"]["df_path"] = {
    "15min": str(DATA_PATH / f"{PAIR}_15min.pkl"),
    "1h": str(DATA_PATH / f"{PAIR}_1h.pkl"),
    "4h": str(DATA_PATH / f"{PAIR}_4h.pkl"),
}
trainer_config

{'env': 'TradingEnv-v1',
 'env_config': {'fee': 0.0001,
  'observer': {'type': 'MultiTimeframeObserver',
   'df_path': {'15min': '/home/napnel/data/BTCUSDT/BTCUSDT_15min.pkl',
    '1h': '/home/napnel/data/BTCUSDT/BTCUSDT_1h.pkl',
    '4h': '/home/napnel/data/BTCUSDT/BTCUSDT_4h.pkl'},
   'window_size': 30},
  'actions': {'type': 'MarketOrder'},
  'rewards': {'type': 'DifferentialSharpRatio', 'window_size': 30},
  'informer': {'type': 'PrivateInformer'},
  'stopper': {'type': 'DrawdownStoper', 'allowable_drawdown': 0.5}},
 'evaluation_config': {'env_config': {'fee': 0.0001,
   'observer': {'type': 'PublicObserver', 'df_path': None, 'window_size': 30},
   'actions': {'type': 'MarketOrder'},
   'rewards': {'type': 'DifferentialSharpRatio', 'window_size': 30},
   'informer': {'type': 'PrivateInformer'},
   'stopper': {'type': 'DrawdownStoper', 'allowable_drawdown': 0.5}},
  'explore': False},
 'evaluation_interval': None,
 'evaluation_duration': 1,
 'evaluation_parallel_to_training': False,

In [11]:
env = TradingEnv(trainer_config["env_config"])

KeyError: 'kwargs'

In [ ]:
raise NotImplementedError

In [ ]:
config = {
    "framework": "torch",
    "env": TradingEnv,
    "model": {
        "custom_model": "MTSFCNetwork",
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
        "post_fcnet_hiddens": [256, 256],
        "post_fcnet_activation": "relu",
    },
    # "rollout_fragment_length": 1,
    # "train_batch_size": 2,
    "num_workers": 0,
    "_disable_preprocessor_api": False,
    # "log_level": "DEBUG",
}

In [ ]:
from ray.rllib.agents import dqn, pg
# agent = dqn.DQNTrainer(config)
agent = pg.PGTrainer(config)

2022-08-19 21:18:08,367	WARNING deprecation.py:46 -- DeprecationWarning: `ray.rllib.agents.pg.default_config::DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.agents.pg.pg.PGConfig(...)` instead. This will raise an error in the future!
2022-08-19 21:18:08,373	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


In [ ]:
# env = SimpleRPG({})
from pprint import pprint
env = SimpleTradingEnv({})
obs = env.reset()
done = False
while not done:
    # action = env.action_space.sample()
    action = agent.compute_action(obs)
    obs, reward, done, info = env.step(action)
    pprint(obs)
    print(action)
    # for k, v in obs.items():
    #     print(k, v)

2022-08-19 21:18:08,879	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


OrderedDict([('long',
              array([[-0.95463943, -1.4243516 ],
       [ 1.0723916 , -1.1243067 ],
       [ 0.9552556 , -0.00147753],
       [ 0.46864143, -0.79806536],
       [-0.34719235,  1.2444327 ]], dtype=float32)),
             ('middle',
              array([[-0.71271336,  1.7044133 ],
       [-0.64632124, -0.6873945 ],
       [ 1.8642262 ,  0.42262438],
       [ 0.5868349 , -0.5430012 ]], dtype=float32)),
             ('position', array([-0.5119855 ,  0.04123053], dtype=float32)),
             ('short',
              array([[-0.04764503,  1.481035  ],
       [ 0.6720204 , -0.1017799 ],
       [-0.7404281 , -0.89726263]], dtype=float32))])
0


In [ ]:
agent.get_policy().model.flatten

{0: FullyConnectedNetwork(
   (_hidden_layers): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=10, out_features=64, bias=True)
         (1): ReLU()
       )
     )
     (1): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=64, out_features=64, bias=True)
         (1): ReLU()
       )
     )
   )
   (_value_branch_separate): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=10, out_features=64, bias=True)
         (1): ReLU()
       )
     )
     (1): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=64, out_features=64, bias=True)
         (1): ReLU()
       )
     )
   )
   (_value_branch): SlimFC(
     (_model): Sequential(
       (0): Linear(in_features=64, out_features=1, bias=True)
     )
   )
 ),
 1: FullyConnectedNetwork(
   (_hidden_layers): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=8, out_features=64, 